# Actividad 1

In [62]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()
print("Valores faltantes por columna:")
print(valores_faltantes)

# Renombrar la columna 'Rotulo_Titulado' a 'Rotulo_Titulo' si es necesario
if 'Rotulo_Titulado' in datos.columns:
    datos = datos.rename(columns={'Rotulo_Titulado': 'Rotulo_Titulo'})

# Verificar si la columna 'Rotulo_Titulo' existe
if 'Rotulo_Titulo' not in datos.columns:
    raise ValueError("La columna 'Rotulo_Titulo' no se encuentra en el conjunto de datos.")

# Codificar la columna objetivo (si es necesario)
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Verificar los valores únicos en la columna objetivo
print("Valores únicos en 'Rotulo_Titulo':", datos['Rotulo_Titulo'].unique())

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Guardar los datos preprocesados para la siguiente actividad
datos_codificados.to_csv('datos_preprocesados.csv', index=False)

print("Preprocesamiento completo y datos guardados en 'datos_preprocesados.csv'.")

Valores faltantes por columna:
Id                                                     0
MAT_1SEM_PROM                                          0
FIS_1SEM_PROM                                          0
ING_1SEM_PROM                                          0
ACTF_1SEM_A                                            0
OTRANS_1SEM_A                                          0
OTRANS_1SEM_R                                          0
OTRANS_1SEM_PROM                                       0
ESP_1SEM_A                                             0
ESP_1SEM_R                                             0
ESP_1SEM_PROM                                          0
INS_1SEM                                               0
PROM_1SEM                                              0
MAT_2SEM_PROM                                          0
FIS_2SEM_PROM                                          0
ING_2SEM_PROM                                          0
ACTF_2SEM_A                                            0


# Actividad 2

In [36]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Cargar el conjunto de datos preprocesado
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Verificar nuevamente el tipo de y después de la codificación
print("Tipos únicos en 'Rotulo_Titulo':", y.unique())

# Definir el número de pliegues para la validación cruzada
num_pliegues = 5
validacion_cruzada = StratifiedKFold(n_splits=num_pliegues, shuffle=True, random_state=42)

# Iterar sobre los pliegues
for i, (train_index, val_index) in enumerate(validacion_cruzada.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    print(f"Pliegue {i+1}: Tamaño de entrenamiento = {len(train_index)}, Tamaño de validación = {len(val_index)}")


Tipos únicos en 'Rotulo_Titulo': [ 0 -1]
Pliegue 1: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 2: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 3: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 4: Tamaño de entrenamiento = 2415, Tamaño de validación = 603
Pliegue 5: Tamaño de entrenamiento = 2415, Tamaño de validación = 603


# Actividad 3

In [49]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Cargar los datos preprocesados
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Definir los hiperparámetros para SVM
parametros_svm = {
    'C': [1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3]
}

# Crear el modelo de SVM
modelo_svm = SVC()

# Ejecutar la búsqueda de hiperparámetros para SVM
grid_search_svm = GridSearchCV(modelo_svm, parametros_svm, cv=5, verbose=1, n_jobs=-1)
grid_search_svm.fit(X, y)

# Mostrar resultados
print("Mejor configuración encontrada para SVM:")
print(grid_search_svm.best_params_)
print("Puntaje de validación cruzada:")
print(grid_search_svm.best_score_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


KeyboardInterrupt: 

In [54]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Cargar los datos preprocesados
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Definir los hiperparámetros para Random Forest
parametros_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

# Crear el modelo de Random Forest
modelo_rf = RandomForestClassifier()

# Ejecutar la búsqueda de hiperparámetros para Random Forest
grid_search_rf = GridSearchCV(modelo_rf, parametros_rf, cv=5, verbose=1, n_jobs=-1)
grid_search_rf.fit(X, y)

# Mostrar resultados
print("Mejor configuración encontrada para Random Forest:")
print(grid_search_rf.best_params_)
print("Puntaje de validación cruzada:")
print(grid_search_rf.best_score_)

from joblib import dump

# Guardar el mejor modelo en un archivo
ruta_modelo = 'mejor_modelo_rf.joblib'
dump(grid_search_rf.best_estimator_, ruta_modelo)

print(f"Mejor modelo guardado correctamente en '{ruta_modelo}'")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejor configuración encontrada para Random Forest:
{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 150}
Puntaje de validación cruzada:
0.7531355364458062
Mejor modelo guardado correctamente en 'mejor_modelo_rf.joblib'


# Actividad 4

In [45]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos
archivo = 'Taller_2_Titulacion_Evaluación.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Guardar los datos preprocesados para la siguiente actividad
datos_codificados.to_csv('datosEvaluacion_preprocesados.csv', index=False)

print("Preprocesamiento completo y datos guardados en 'datosEvaluacion_preprocesados.csv'.")

Preprocesamiento completo y datos guardados en 'datosEvaluacion_preprocesados.csv'.


In [66]:
import pandas as pd
from joblib import load
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos de evaluación
archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

# Cargar el mejor modelo que encontraste en la actividad anterior
mejor_modelo = load('mejor_modelo_rf.joblib')

# Preprocesar los datos de evaluación de la misma manera que hiciste en actividades previas (si es necesario)

# Ver el número de valores faltantes por columna
valores_faltantes = datos_evaluacion.isnull().sum()

# Identificar las columnas categóricas
columnas_categoricas = datos_evaluacion.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_evaluacion_codificados = pd.get_dummies(datos_evaluacion, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_evaluacion_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_evaluacion_codificados[columnas_numericas] = escalador.fit_transform(datos_evaluacion_codificados[columnas_numericas])
print("Columnas en X_evaluacion:", list(X_evaluacion.columns))
print("Columnas del modelo:", list(mejor_modelo.feature_names_in_))

# Realizar predicciones
X_evaluacion = datos_evaluacion_codificados.drop('Id', axis=1)  # Ajustar esto según la estructura de tus datos de evaluación
predicciones = mejor_modelo.predict(X_evaluacion)

# Preparar el archivo de salida
resultados = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Prediccion': predicciones})
resultados['Prediccion'] = resultados['Prediccion'].apply(lambda x: 'SÍ' if x == 1 else 'NO')

# Guardar los resultados en un archivo de texto
archivo_salida = 'clasificación_título4.csv'
resultados.to_csv(archivo_salida, sep=',', index=False)

print(f"Archivo de clasificación generado correctamente: '{archivo_salida}'")


Columnas en X_evaluacion: ['MAT_1SEM_PROM', 'FIS_1SEM_PROM', 'ING_1SEM_PROM', 'ACTF_1SEM_A', 'ACTF_1SEM_R', 'OTRANS_1SEM_A', 'OTRANS_1SEM_R', 'OTRANS_1SEM_PROM', 'ESP_1SEM_A', 'ESP_1SEM_R', 'ESP_1SEM_PROM', 'INS_1SEM', 'PROM_1SEM', 'MAT_2SEM_PROM', 'FIS_2SEM_PROM', 'ING_2SEM_PROM', 'ACTF_2SEM_A', 'ACTF_2SEM_R', 'OTRANS_2SEM_A', 'OTRANS_2SEM_R', 'OTRANS_2SEM_PROM', 'ESP_2SEM_A', 'ESP_2SEM_R', 'ESP_2SEM_PROM', 'INS_2SEM', 'PROM_2SEM', 'INSC_AnO', 'PROM_AnO', 'numero_horario_trabajo', 'inferior_ingreso_bruto_fam', 'superior_ingreso_bruto_fam', 'horas_promedio_trabajo', 'cantidad_personas_grupo_familiar', 'cuantos_trabajan_grupo_familiar', 'cuantos_estudian_grupo_pre_basica', 'cuantos_estudian_grupo_basica', 'cuantos_estudian_grupo_media_1_3', 'cuantos_estudian_grupo_media_4', 'cuantos_estudian_grupo_superior', 'cuantos_estudian_grupo_otras', 'ptje_nem', 'ptje_leng', 'ptje_mate', 'ptje_hycs', 'ptje_cien', 'GENERO_M', 'NOM_CARRERA_Ingeniería de Ejecución en Prevención De Riesgos', 'NOM_CARR

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ACTF_1SEM_R
- ACTF_2SEM_R
- cuantos_estudian_grupo_basica
- cuantos_estudian_grupo_media_1_3
- cuantos_estudian_grupo_media_4
- ...
Feature names seen at fit time, yet now missing:
- Id
- descripcion_situacion_ocupacional_padre_Dueno de casa
- nombre_comuna_EM_ALGARROBO
- nombre_comuna_EM_ALHUE
- nombre_comuna_EM_ALTO HOSPICIO
- ...
